**LANE DETECTION PROJECT**

Rahul Vimalkanth

In [ ]:
# importing libraries
import cv2 as cv
import numpy as np
from IPython.display import Video

In [ ]:
#creating a region of interest by masking other regions
def create_roi(frame, vertices):
    mask = np.zeros_like(frame)
    cv.fillPoly(mask, [vertices], 255)
    masked_img = cv.bitwise_and(frame, mask)
    return masked_img

Canny Edge Detection Algorithm

1. Noise Reduction: The first step is to reduce noise in the image using a Gaussian blur. This helps to smooth out the image and remove high-frequency noise, which can cause false edge detection.

2. Gradient Calculation: Next, the gradient (rate of change) of pixel intensity at each point in the image is calculated. This is typically done using derivatives (such as the Sobel operator) to find the rate of change in pixel intensity in both the x and y directions.

3. Edge Strength and Orientation: From the gradients, the edge strength (magnitude) and orientation (direction) are calculated for each pixel. The edge strength represents the rate of change in intensity, while the orientation indicates the direction of the gradient.

4. Non-maximum Suppression: This step involves thinning the edges by suppressing all gradient values except for those that represent local maxima along the direction of the gradient. This helps to ensure that only the most significant edges are retained.

5. Edge Hysteresis Thresholding: Finally, the edges are further refined by applying two thresholds: a high threshold (above which pixels are considered strong edges) and a low threshold (below which pixels are considered non-edges). Pixels with gradient magnitudes above the high threshold are considered strong edges, while pixels between the low and high thresholds are considered weak edges. Weak edges are retained only if they are connected to strong edges, forming continuous edge contours.

The output of the Canny edge detection algorithm is a binary edge map where pixels corresponding to edges are set to white (255) and non-edge pixels are set to black (0)

In [ ]:
# running canny edge detection
def pre_process(img):
    roi_vertices = np.array([(200, 700), (1100, 700), (750, 400), (550, 400)], np.int32)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    canny = cv.Canny(gray, 100, 200)
    cropped = create_roi(canny, roi_vertices)
    return cropped

cv.addWeighted()

image: This is the first input image.

line_image: This is the second input image.

0.8: The weight (alpha) for the first image. It determines how much the first image contributes to the final result.

1: The weight (beta) for the second image. It determines how much the second image contributes to the final result.

0: The gamma value, which is an optional parameter. It controls the brightness of the output image.

The resulting final_image will be a combination of the two input images, with the specified weights. It’s commonly used for overlaying one image on top of another, such as adding lines or annotations to an existing image.

In [ ]:
# drawing lines on detected edges
def draw_lines(img, lines):
    img = np.copy(img)
    blank_image = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                cv.line(blank_image, (x1, y1), (x2, y2), (0, 0, 255), thickness=10)
    img = cv.addWeighted(img, 0.8, blank_image, 1, 0.0)
    return img

In [ ]:
# capturing and displaying the video
cap = cv.VideoCapture("Videos/Computer_Vision/highway_-_10364 (720p).mp4")
Video("Videos/Computer_Vision/highway_-_10364 (720p).mp4")

The Probabilistic Hough Line Transform is an improved version of the standard Hough Line Transform.
It is used for detecting straight lines in an image.The probabilistic variant directly provides the endpoints of the detected line segments.

Adjusting the parameters of the Hough Line Transform can significantly impact the accuracy and robustness of lane detection.

rho: Distance resolution of the accumulator in pixels. Decreasing rho increases the resolution, potentially detecting smaller lines, but it also increases computational load.

theta: Angular resolution of the accumulator in radians. Smaller values allow for more precision but also increase computation.

threshold: Minimum number of votes (intersections in Hough grid cell) required for a line to be detected. Increase to make detection stricter, reducing false positives, but be cautious not to filter out valid lines.

minLineLength: Minimum length of a line (in pixels) to be considered a line. Increasing this value filters out shorter lines, reducing noise but potentially missing valid shorter lines.

maxLineGap: Maximum allowed gap between line segments to be considered as a single line. Increasing this value allows for gapped lines to be connected, potentially detecting longer lines but also increasing the risk of merging unrelated segments.

By using cap.set(cv.CAP_PROP_POS_FRAMES, 0), we ensure that if the video capture object reaches the end of the video (i.e., ret becomes False), it will reset the current position to the start of the video, allowing the video processing loop to continue capturing frames from the beginning. This ensures continuous looping through the video without interruption

In [ ]:
while cap.isOpened():
    ret, frame = cap.read() #reading the video
    if not ret:
        cap.set(cv.CAP_PROP_POS_FRAMES, 0)
        continue
    modified_img = pre_process(frame)
    lines = cv.HoughLinesP(modified_img, rho=3, theta=np.pi/180, threshold=45, minLineLength=25, maxLineGap=35)
    img = draw_lines(frame, lines)
    cv.imshow('Lane Detection', img)
    cv.imshow('Edge',modified_img)
    
    # Check for 'esc' key press to exit
    key = cv.waitKey(10)
    if key == 27:  
        break

# Release the video capture and close all windows
cap.release()
cv.destroyAllWindows()